In [1]:
import numpy as np
from scipy.sparse.linalg import svds, LinearOperator

In [2]:
import torch

## Cola

In [2]:
import pickle
with open("grads5_11.pickle", "rb") as fp:   #Pickling
    dict_of_grads = pickle.load(fp)

In [3]:
dict_of_grads.keys()

dict_keys(['bert.encoder.layer.8.intermediate.dense', 'bert.encoder.layer.8.output.dense', 'bert.encoder.layer.9.intermediate.dense', 'bert.encoder.layer.9.output.dense', 'bert.encoder.layer.10.intermediate.dense', 'bert.encoder.layer.10.output.dense', 'bert.encoder.layer.11.intermediate.dense', 'bert.encoder.layer.11.output.dense', 'bert.encoder.layer.5.output.dense', 'bert.encoder.layer.6.output.dense', 'bert.encoder.layer.5.intermediate.dense', 'bert.encoder.layer.6.intermediate.dense', 'bert.encoder.layer.7.intermediate.dense', 'bert.encoder.layer.7.output.dense'])

In [4]:
#new_dict_of_grads = {}

In [17]:
list_of_desired_grad = ['bert.encoder.layer.5.intermediate.dense', 'bert.encoder.layer.6.intermediate.dense','bert.encoder.layer.7.intermediate.dense','bert.encoder.layer.6.output.dense']

In [14]:
len(new_dict_of_grads['bert.encoder.layer.5.output.dense'])

67

In [20]:
for k, v in dict_of_grads.items():
    if k in ['bert.encoder.layer.7.output.dense']:
        new_dict_of_grads[k] = v 

In [ ]:
import os
with open(os.path.join('grads5_11.pickle'), 'wb') as f:
        pickle.dump(new_dict_of_grads, f)

In [1]:
new_dict_of_grads.keys()

NameError: name 'new_dict_of_grads' is not defined

In [11]:
import pickle
with open("tensor_grad8_11.pickle", "rb") as fp:   #Pickling
    dict_of_grads = pickle.load(fp)

list_of_grads = dict_of_grads['bert.encoder.layer.7.intermediate.dense']

EOFError: Ran out of input

In [5]:
m, n = list_of_grads[0].shape #cols rows in torch -> rows cols in numpy
print ("m, n", m, n)
list_of_grads1 = [grad.reshape(-1).numpy() for grad in list_of_grads]

grad_vectors = np.stack([grad.reshape(n,m, order = 'F') for grad in list_of_grads1])

m, n 3072 768


In [6]:
grad_vectors[0].shape

(768, 3072)

In [7]:
def matvec(vec):
    print ('matvec')
    k, m, n = grad_vectors.shape
    V = vec.reshape(m, m, order='F')
    res = np.zeros(n*n) 
    for i in range(k):
        res += (grad_vectors[i].T @ V @ grad_vectors[i]).T.ravel()
    return res

def r_matvec(vec):
    print ('rmatvec')
    k, m, n = grad_vectors.shape
    V = vec.reshape(n, n, order='F')
    res = np.zeros(m*m) 
    for i in range(k):
        res += (grad_vectors[i] @ V @ grad_vectors[i].T).T.ravel()
    return res

In [ ]:
linop_m = LinearOperator(
    shape=(m**2, n**2),
    matvec=matvec,
    rmatvec=r_matvec
)

# Compute SVD
print("Performing SVD...", flush=True)
u, s, vt = svds(linop_m, k = 3, return_singular_vectors=True)
sidx = np.argsort(-s)
s = s[sidx]
u = u[:, sidx]
v = vt[sidx, :].T

matvec
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
Performing SVD...
matvec
[-0.00095379  0.00162867  0.00021598 ...  0.00463541  0.00087444
 -0.02302475]
[-7.11780411e-06  1.21542403e-05  1.61181578e-06 ...  3.45925987e-05
  6.52567964e-06 -1.71826500e-04]
rmatvec
matvec
[-0.00186252  0.00081314 -0.00010622 ...  0.00214817  0.00043681
 -0.01127561]
[-1.38993760e-05  6.06818438e-06 -7.92714915e-07 ...  1.60311360e-05
  3.25975596e-06 -8.41463250e-05]
rmatvec
matvec
[-3.65603833e-04 -1.04098500e-04 -4.88578659e-05 ...  4.35803640e-05
  3.24285211e-05 -1.74777353e-04]
[-2.72838682e-06 -7.76854478e-07 -3.64610940e-07 ...  3.25226597e-07
  2.42003889e-07 -1.30430860e-06]
rmatvec
matvec
[ 3.69676489e-04  7.91645882e-05  8.49818236e-05 ... -6.91346828e-05
  4.61466388e-05 -5.71673731e-06]
[ 2.75877977e-06  5.90780509e-07  6.34192713e-07 ... -5.15930469e-07
  3.44377902e-07 -4.26622187e-08]
rmatvec
matvec
[ 2.80559360e-04  8.07779314e-05  5.62485471e-05 ... -1.74079459e-04
 -2.71258238e-0

In [10]:
s

array([0.04702242, 0.0142314 , 0.01274068])

In [14]:
x_ = u[:, 0] * s[0]
y_ = v[:, 0]
XF = x_.reshape(m, m, order='F')
YF = y_.reshape(n, n, order='F')

In [15]:
import numpy as np

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

is_pos_def(YF)

True

In [17]:
np.save("C1_7sgd_norm.npy", XF)
np.save("B1_7sgd_norm.npy", YF)